In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 0.001

img_transform = transforms.Compose([transforms.ToTensor()])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    
    # Defining the network
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 5, stride=1, padding=1), nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 8, 5, stride=1, padding=1), nn.MaxPool2d(2, stride=2),
            nn.Conv2d(8, 4, 2, stride=1, padding=1), nn.MaxPool2d(2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.MaxUnpool2d(2, stride=2), nn.ConvTranspose2d(4, 8, 2, stride=1),
            nn.MaxUnpool2d(2, stride=2), nn.ConvTranspose2d(8, 16, 5, stride=1),
            nn.MaxUnpool2d(2, stride=2), nn.ConvTranspose2d(16, 1, 5, stride=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Getting our model
model = autoencoder()

# Defining optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00001)
criterion = nn.MSELoss()


for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img)
        
        # Forward Propagation
        output = model(img)
        loss = criterion(output, img)
        
        # Back Propagation
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    
    # Accuracy
    print('epoch {} = loss:{:.4f}'
          .format(epoch+1, loss.data))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder.pth')

TypeError: forward() missing 1 required positional argument: 'indices'

NameError: name 'img' is not defined